In [1]:
import requests
import os
from dotenv import load_dotenv

## Import environment variables

In [2]:
# Import environment variables from app_secrets.env
load_dotenv(dotenv_path="../secrets/app_secrets.env", verbose=True)
# Bearer token for authentication
INBOUND_CHECK_TOKEN = os.getenv("INBOUND_CHECK_TOKEN")
# Hosts
local_host = os.getenv("LOCAL_HOST")  # mc_core local
dev_host = os.getenv("DEV_HOST")  # mc_core dev
staging_host = os.getenv("STAGING_HOST")  # mc_core staging

Set host to use for notebook:

In [3]:
host = staging_host
bearer_token = "Bearer " + INBOUND_CHECK_TOKEN

# inbound/check
Variables from this cell will be used for the other two endpoints

In [4]:
#### Test the check endpoint
response = requests.post(
    host + "/inbound/check",
    json={"text_to_match": "test message"},
    headers={"Authorization": bearer_token},
)

#### store info from response
top_response = response.json()["top_responses"][0][0]
# for feedback endpoint
inbound_id = response.json()["inbound_id"]
feedback_secret_key = response.json()["feedback_secret_key"]
# for pagination endpoint (also inbound_id above)
inbound_secret_key = response.json()["inbound_secret_key"]
initial_next_page_url = response.json()["next_page_url"]

#### print results
print("top response:", top_response)
print("check endpoint response time (s):", response.elapsed.total_seconds())

top response: HIV - Testing
check endpoint response time (s): 0.767358


In [5]:
# response.json()

# inbound/feedback

In [6]:
#### Simple function to test the feedback endpoint
def feedback_endpoint_test(inbound_id, feedback_secret_key, feedback):

    feedback_response = requests.put(
        host + "/inbound/feedback",
        json={
            "inbound_id": inbound_id,
            "feedback_secret_key": feedback_secret_key,
            "feedback": feedback,
        },
        headers={"Authorization": bearer_token},
    )

    print("Feedback call response:", feedback_response.text)
    print("Feedback endpoint response time (s):", response.elapsed.total_seconds())

In [7]:
# correct information
feedback_endpoint_test(
    inbound_id=inbound_id,
    feedback_secret_key=feedback_secret_key,
    feedback={"feedback_type": "positive", "faq_id": 0}, #or "page_number": 0. See below.
)

Feedback call response: Success
Feedback endpoint response time (s): 0.767358


In [8]:
# correct information
feedback_endpoint_test(
    inbound_id=inbound_id,
    feedback_secret_key=feedback_secret_key,
    feedback={"feedback_type": "positive", "page_number": 0},
)

Feedback call response: Malformed Feedback JSON
Feedback endpoint response time (s): 0.767358


In [9]:
# incorrect feedback format
feedback_endpoint_test(
    inbound_id=inbound_id, feedback_secret_key=feedback_secret_key, feedback="Gold Star"
)

Feedback call response: Malformed Feedback JSON
Feedback endpoint response time (s): 0.767358


In [10]:
# incorrect inbound_id
feedback_endpoint_test(
    inbound_id=0,
    feedback_secret_key=feedback_secret_key,
    feedback={"feedback_type": "positive", "faq_id": 0}, # or "page_number": 0
)

Feedback call response: No Matches
Feedback endpoint response time (s): 0.767358


In [11]:
# incorrect feedback_secret_key
feedback_endpoint_test(
    inbound_id=inbound_id,
    feedback_secret_key="abcdefg",
    feedback={"feedback_type": "positive", "faq_id": 0}, # or "page_number": 0
)

Feedback call response: Incorrect Feedback Secret Key
Feedback endpoint response time (s): 0.767358


# Test pagination 
#### inbound/inbound_id/page_number

There are two approaches here:
1. Jumping to a page using the page number
2. Using the next and previous page urls returned by each call to flip through pages

## Jump-to-page

In [13]:
# Simple function to jump to a page and return content
def jump_to_page(page_number, inbound_secret_key):

    # call
    response = requests.get(
        url=host + "/inbound/" + inbound_id + "/" + str(page_number),
        params={
            "inbound_secret_key": inbound_secret_key
        },  #!# params, NOT json. Opposite of other endpoint. #!#
        headers={"Authorization": bearer_token},
    )

    # show results
    if response.status_code != 200:
        print("Error:", response.status_code, response.text)
    else:
        if response.json()["top_responses"] == []:
            print("No content.")
        else:
            print("top response:", response.json()["top_responses"][0][0])
        print("check endpoint response time (s):", response.elapsed.total_seconds())
        if "prev_page_url" in response.json():
            print("Previous page url:", "found")
        if "next_page_url" in response.json():
            print("Next page:", "found")

### Page 1 should match initial call response

In [14]:
jump_to_page(1, inbound_secret_key)

top response: HIV - Testing
check endpoint response time (s): 0.664992
Next page: found


### Page 2 should be different

In [15]:
jump_to_page(2, inbound_secret_key)

top response: Support - HIV
check endpoint response time (s): 0.686843
Previous page url: found
Next page: found


### !!!! Go to non-existing page !!!!
Note: Should probably throw an error if a non-existing page is attempted to be accessed? Currently returns empty response but also returns a previous page link (which also can lead to an empty page) but no next page link.

In [18]:
jump_to_page(-1, inbound_secret_key)

Error: 404 <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 3.2 Final//EN">
<title>404 Not Found</title>
<h1>Not Found</h1>
<p>The requested URL was not found on the server. If you entered the URL manually please check your spelling and try again.</p>



In [19]:
jump_to_page(0, inbound_secret_key)

No content.
check endpoint response time (s): 0.827296
Next page: found


In [20]:
jump_to_page(999, inbound_secret_key)

No content.
check endpoint response time (s): 0.719756
Previous page url: found


## Flip pages (next and previous)

### Next page from initial call should be the same as page 2

In [21]:
#### Test pagination endpoint - flip pages
response = requests.get(
    url=host + initial_next_page_url,
    headers={"Authorization": bearer_token},
)

# show results
print("top response:", response.json()["top_responses"][0][0])
print("check endpoint response time (s):", response.elapsed.total_seconds())

top response: Support - HIV
check endpoint response time (s): 0.714804


### Flip through all available pages

In [22]:
url = initial_next_page_url
page_number = 2

while True:

    # print current page number
    print("Page", page_number)

    #### call
    response = requests.get(
        url=host + url,
        headers={"Authorization": bearer_token},
    )

    #### print results
    print("top response:", response.json()["top_responses"][0][0])
    print("check endpoint response time (s):", response.elapsed.total_seconds())

    #### return next/prev page link
    # Note: The response will simply not include the next/prev page url field if it
    # doesn't exist, resulting in an error if we try to select it from the JSON response.
    if "next_page_url" in response.json():
        url = response.json()["next_page_url"]
        page_number += 1
    else:
        print("ERROR: No next/prev page.")
        break

Page 2
top response: Support - HIV
check endpoint response time (s): 0.661246
Page 3
top response: HIV - Symptoms
check endpoint response time (s): 0.631793
Page 4
top response: Preg - Maternity leave
check endpoint response time (s): 0.803331
Page 5
top response: Baby - Growth - Birth to 3 months
check endpoint response time (s): 0.685865
Page 6
top response: Privacy - Who can see my personal
check endpoint response time (s): 0.683107
Page 7
top response: Baby - Returning period
check endpoint response time (s): 0.673245
Page 8
top response: Preg - Position at birth
check endpoint response time (s): 0.688149
Page 9
top response: Preg - Fetal Month 9
check endpoint response time (s): 0.680234
Page 10
top response: HIV - Nevirapine syrup
check endpoint response time (s): 0.770041
Page 11
top response: COVID-19 - Breastfeeding
check endpoint response time (s): 0.677768
Page 12
top response: Labour - Stages
check endpoint response time (s): 0.65873
Page 13
top response: Support - Abortion